This notebook will be used for the Capstone Project for the course offered on Coursera

In [1]:
#importing required libraries
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

print("Hello Capstone Project Course!")
print("Libraries successfully imported.")

Hello Capstone Project Course!
Libraries successfully imported.


In [104]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import ssl
import csv

print('Soup & CSV successfully imported.')

# Ignore SSL certificate errors
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(source, 'lxml')

print('soup ready')

Soup & CSV successfully imported.
soup ready


In [105]:
table = soup.find('table', {'class':'wikitable sortable'})

In [106]:
tableRows = table.find_all('tr')

In [107]:
data = []
for row in tableRows:
    data.append([t.text.strip() for t in row.find_all('td')])

df = pd.DataFrame(data, columns=['PostalCode', 'Borough', 'Neighbourhood'])
df = df[~df['PostalCode'].isnull()]  # to filter out bad rows

print(df.head(5))
df.info()
df.shape

  PostalCode           Borough               Neighbourhood
1        M1A      Not assigned                            
2        M2A      Not assigned                            
3        M3A        North York                   Parkwoods
4        M4A        North York            Victoria Village
5        M5A  Downtown Toronto  Regent Park / Harbourfront
<class 'pandas.core.frame.DataFrame'>
Int64Index: 180 entries, 1 to 180
Data columns (total 3 columns):
PostalCode       180 non-null object
Borough          180 non-null object
Neighbourhood    180 non-null object
dtypes: object(3)
memory usage: 5.6+ KB


(180, 3)

In [108]:
#Now the dataframe cleaning part
df.drop(df[df['Borough']=="Not assigned"].index,axis=0, inplace=True)
print(df.shape)

df.head()

(103, 3)


,PostalCode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Regent Park / Harbourfront
6,M6A,North York,Lawrence Manor / Lawrence Heights
7,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [109]:
df.loc[df['Neighbourhood']=="Not assigned",'Neighbourhood']=df.loc[df['Neighbourhood']=="Not assigned",'Borough']

In [110]:
df.head()

,PostalCode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Regent Park / Harbourfront
6,M6A,North York,Lawrence Manor / Lawrence Heights
7,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [111]:
dfTor = df.groupby('PostalCode').agg(lambda x: ','.join(x))
dfTor.head()

,Borough,Neighbourhood
PostalCode,,
M1B,Scarborough,Malvern / Rouge
M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
M1E,Scarborough,Guildwood / Morningside / West Hill
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


In [112]:
dfToronto = dfTor.reset_index()
dfToronto['Borough']= dfToronto['Borough'].str.replace('nan|[{}\s]','').str.split(',').apply(set).str.join(',').str.strip(',').str.replace(",{2,}",",")

In [113]:
dfToronto.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [114]:
dfToronto.shape

(103, 3)

In [125]:
#installing Geopy & Geocoder
!pip install geopy
!pip install geocoder
print("Geopy & Geocoder has been installed")

     |████████████████████████████████| 102kB 16.9MB/s ta 0:00:01
Geopy & Geocoder has been installed


In [117]:
from  geopy.geocoders import Nominatim
geolocator = Nominatim()
neighborhood = "Toronto, North York, Parkwoods"
loc = geolocator.geocode(neighborhood)
print("latitude is :-" ,loc.latitude,"\nlongtitude is:-" ,loc.longitude)

latitude is :- 43.7587999 
longtitude is:- -79.3201966


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  from ipykernel import kernelapp as app


In [118]:
dfGeopy = pd.DataFrame({'PostalCode': ['M3A', 'M4A', 'M5A'],
                         'Borough': ['North York', 'North York', 'Downtown Toronto'],
                         'Neighbourhood': ['Parkwoods', 'Victoria Village', 'Harbourfront'],})

In [123]:
dfGeopy = dfToronto
dfGeopy['address'] = dfToronto[['PostalCode', 'Borough', 'Neighbourhood']].apply(lambda x: ', '.join(x), axis=1 )
dfGeopy.shape

(103, 4)

In [124]:
dfGeopy.drop(dfGeopy[dfGeopy['Borough']=="Notassigned"].index,axis=0, inplace=True)
# code holds true up until i=102
dfGeopy.shape

(103, 4)